# Summarization of financial data using a Large Language Model (LLM)

This notebook aims to provide an introduction to documenting an LLM using the ValidMind Developer Framework. The use case presented is a summarization of financial news (https://huggingface.co/datasets/cnn_dailymail).

- Initializing the ValidMind Developer Framework
- Running a test various tests to quickly generate documentation about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

To access the [ValidMind Platform UI](https://app.prod.validmind.ai), you'll need an account. If you're new, you can create your account the first time you log in. Signing up is FREE.

This notebook requires an OpenAI API secret key to run. If you don't have one, visit [API keys](https://platform.openai.com/account/api-keys) on OpenAI's site to create a new key for yourself. Note that API usage charges may apply.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [ ]:
%pip install -q validmind

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Foundation Model - Text Summarization`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace the code below with the code snippet from your project ##

import validmind as vm

vm.init(
    api_host = "https://api.prod.validmind.ai/api/v1/tracking",
    api_key = "...",
    api_secret = "...",
    project = "..."
)

## Load data

### CNN dataset

The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

In [ ]:
from datasets import load_dataset

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')

train_df = cnn_dataset.data['train'].to_pandas()
test_df = cnn_dataset.data['test'].to_pandas()
val_df = cnn_dataset.data['validation'].to_pandas()

train_df = train_df[['article','highlights']]
test_df = test_df[['article','highlights']]

train_df = train_df.head(5)
test_df = test_df.head(5)

## Model selection

In [ ]:
from validmind.models import FoundationModel, Prompt
import os
import dotenv
import openai

dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")


def call_model(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message["content"]

## Prompt engineering

In [ ]:
prompt_template = """
You are an AI with expertise in summarizing financial news.
Your task is to provide a concise summary of the specific news article provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the article.

Article to Summarize:

```
{article}
```

Please respond with a concise summary of the article's main points.
Ensure that your summary is based on the content of the article and not on external information or assumptions.
""".strip()

prompt_variables = ["article"]

## Initialize ValidMind datasets and models

In [ ]:
vm_test_ds = vm.init_dataset(
    dataset=test_df,
    text_column="article",
    target_column="highlights",
)

vm_model = FoundationModel(
    predict_fn=call_model,
    prompt=Prompt(
        template=prompt_template,
        variables=prompt_variables,
    ),
    test_ds=vm_test_ds,
)

## Model performance metrics

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.RougeMetricsAggregate",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.BertScoreAggregate",
    model=vm_model)

## Bias metrics

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.ToxicityHistogram",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.ToxicityScore",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.RegardScore",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.RegardHistogram",
    model=vm_model)